# Подготовительный этап

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from functools import partial

# библиотека для работы с кодировщиком
import category_encoders as ce

# библиотеки для получения координат
import geopy.geocoders
from geopy.geocoders import Nominatim

#отключаем таймаут
geopy.geocoders.options.default_timeout = None 

# библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# инструменты для оценки точности модели
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install geopy
#!pip install category_encoders

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [2]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.info()

In [ ]:
sample_submission.head(2)

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем датасеты

In [ ]:
data.nunique(dropna=False)

# Обработка данных

## Удаление дубликатов

In [ ]:
print(f'Число записей: {data.shape[0]}')
dupl_columns = list(data.columns)

mask_d = data.duplicated(subset=dupl_columns)
data_duplicates = data[mask_d]
print(f'Число найденных дубликатов: {data_duplicates.shape[0]}')

data_dedupped = data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {data_dedupped.shape[0]}')

## Обработка hotel_address

In [ ]:
data['hotel_country'] = data['hotel_address'].apply(lambda x: 'United Kingdom' if x.endswith('United Kingdom') else x.split()[-1])

def get_city(vec):
    address = vec[0]
    country = vec[1]
    address_list = address.split()
    if country == 'United Kingdom':
        city = address_list[-5]
    else:
        city = address_list[-2]
    return city
    
data['hotel_city'] = data[['hotel_address','hotel_country']].apply(get_city, axis=1)

## Восстанавливаем пропуски lat/lng

In [ ]:
def geocoder(addr):
    address = addr[0]
    country = addr[1]
    city = addr[2]
    location = geocode(address)
    #если не удалось вычислить, то берем координаты от строки "город + страна"
    if location == None:
        location = geocode(city + ', ' + country)
    return location

geolocator = Nominatim(user_agent="kaggle_ml")
geocode = partial(geolocator.geocode, language="en")

In [ ]:
mask = data['lat'].isnull()
cols = ['hotel_address','hotel_country','hotel_city']

#выделяем адреса в отдельный датасет
geo = data[mask][cols].reset_index(drop=True)
#оставляем уникальные адреса
geo = geo.drop_duplicates(subset=cols).reset_index(drop=True)

#вычисляем адрес
geo['location'] = geo[cols].apply(geocoder, axis=1)

#вычисляем широту/долготу
geo['latitude'] = geo['location'].apply(lambda x: x.latitude if x else None)
geo['longitude'] = geo['location'].apply(lambda x: x.longitude if x else None)

geo = geo.drop(['hotel_country', 'hotel_city','location'], axis = 1)

#объединяем датасеты
data = data.merge(geo,
    on = 'hotel_address',
    how = 'left'
)

#восстанавливаем пропуски вычисленными данными
data['lat'] = data['lat'].fillna(data['latitude'])
data['lng'] = data['lng'].fillna(data['longitude'])

#удаляем лишние признаки
data.drop(['latitude','longitude','hotel_address'],axis=1, inplace=True) #удаляем признак

## Обработка review_date

In [ ]:
# конвертируем дату
data['review_date'] = pd.to_datetime(data['review_date'])

# создаем новые признаки
data['review_year'] = data['review_date'].dt.year
data['review_month'] = data['review_date'].dt.month
data['review_day_of_week'] = data['review_date'].dt.dayofweek

#удаляем признак
data = data.drop(['review_date'], axis = 1)

## Кодируем название отеля

In [ ]:
# число отелей превышает 15, поэтому используем бинарное кодирование
bin_encoder = ce.BinaryEncoder(cols=['hotel_name'])
type_bin = bin_encoder.fit_transform(data['hotel_name'])
data = pd.concat([data, type_bin], axis=1)

data = data.drop(['hotel_name'], axis = 1)

## Кодируем hotel_city

In [ ]:
# число уникальных городов не превышает 15, поэтому можем использовать OneHotEncoder
encoder = ce.OneHotEncoder(cols=['hotel_city'])
type_bin = encoder.fit_transform(data['hotel_city'])
data = pd.concat([data, type_bin], axis=1)

## Кодировка review_month и review_day_of_week

In [ ]:
# кодировка дня недели методом однократного кодирования OneHotEncoder
encoder = ce.OneHotEncoder(cols=['review_day_of_week'])
type_bin = encoder.fit_transform(data['review_day_of_week'])
data = pd.concat([data, type_bin], axis=1)

# кодировка месяца методом однократного кодирования OneHotEncoder
encoder = ce.OneHotEncoder(cols=['review_month'])
type_bin = encoder.fit_transform(data['review_month'])
data = pd.concat([data, type_bin], axis=1)

# удаляем признак даты отзыва и координаты
data.drop(['review_month', 'review_day_of_week'],axis=1,inplace=True)

## Обработка reviewer_nationality

In [ ]:
# удаляем лишние пробелы
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x.strip())

# совпадают является ли рецензент гражданином страны, в которой находится отель
data['is_citizen'] = data['reviewer_nationality'] == data['hotel_country']
data['is_citizen'] = data['is_citizen'].astype('int')

# выбираем 15 самых популярных национальностей, остальным оставим 'other'
top_nations = list(data['reviewer_nationality'].value_counts()[:15].index)
data['reviewer_nationality'] = data['reviewer_nationality'].apply(lambda x: x if x in top_nations else 'other')

In [ ]:
# кодируется признак национальности рецензента методом двоичного кодирования
bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality'])
type_bin = bin_encoder.fit_transform(data['reviewer_nationality'])
data = pd.concat([data, type_bin], axis=1)

# удаляем преобразованные признаки
data.drop(['hotel_city','hotel_country','reviewer_nationality'],axis=1,inplace=True)

## Обработка tags

In [ ]:
def get_days(tag): 
    # Функция выбирает из тегов количество дней проживания или выдаёт ноль
    days = tag[3:-3].split(" ', ' ")    
    for i in range(len(days)):  
        if days[i].split()[0] == "Stayed":
            return int(days[i].split()[1])
    return 0

# Создаётся новый признак: days - количество дней проживания. 
data['days'] = data['tags'].apply(get_days)

# двоичное кодирование тегов
bin_encoder = ce.BinaryEncoder(cols=['tags'])
type_bin = bin_encoder.fit_transform(data['tags'])
data = pd.concat([data, type_bin], axis=1)

# Отбираем признаки

In [ ]:
corr = data.drop(['sample'], axis=1).corr()

# выведем только данные с высоким уровнем корреляции
mask_1 = np.abs(corr) >= 0.6
mask_2 = np.abs(corr) <= -0.6

corr_data = corr[mask_1 | mask_2]

plt.rcParams['figure.figsize'] = (25,20)

#fmt отвечает за количество знаков после точки
sns.heatmap(corr_data, annot=True, fmt='.1g', vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')

In [ ]:
# Удаляем признаки "object" и "category".
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

# Также убираем признак с высокой кореляцией: 
data.drop(['total_number_of_reviews', 'lat', 'lng'],axis=1,inplace=True)

## Машинное обучение

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга отелея в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

## Вычисляем метрику MAPE

In [ ]:
def mape_func(Y_actual, Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

print('MAPE:', mape_func(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)